In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http:

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [10]:
more_reviews_df = vine_df.filter(vine_df.total_votes>"19")
more_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [11]:
percentage_reviews_df = more_reviews_df.withColumn("review_percentage", more_reviews_df.helpful_votes/more_reviews_df.total_votes)
percentage_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   review_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|0.045454545454545456|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|                 0.8|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|  0.4857142857142857|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N| 0.47058823529411764|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|  0.6774193548387096|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|  0.4482758620689655|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|  0.9642857142857143|
| R33IFUWC

In [12]:
##Filter by reviews that had 50% or greater helpful votes out of total votes
helpful_votes_df = percentage_reviews_df.filter(percentage_reviews_df.review_percentage>=".5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|               0.8|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|0.6774193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|0.9642857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|               1.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|0.8729641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|0.8888888888888888|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|0.9583333333333334|
|R2JSWNTAO83X3V|          5|  

In [18]:
#Retrieve all rows where a review was written as part of the vine program
from pyspark.sql.functions import col
vine_program_df = helpful_votes_df.filter(col("vine") == "Y")
vine_program_df.show()

+---------+-----------+-------------+-----------+----+-----------------+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_percentage|
+---------+-----------+-------------+-----------+----+-----------------+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+-----------------+



In [19]:
#Retrieve all rows where a review was written as not part of the vine program
nonvine_program_df = helpful_votes_df.filter(helpful_votes_df.vine == "N")
nonvine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|               0.8|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|0.6774193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|0.9642857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|               1.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|0.8729641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|0.8888888888888888|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|0.9583333333333334|
|R2JSWNTAO83X3V|          5|  

In [20]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
from pyspark.sql.functions import col,when,count,lit
ratings_total_df = helpful_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total")).show()

+----+-------------+--------------------+-----------------+
|vine|Total_Reviews|Total_5_Star_Reviews|%_5_Star_To_Total|
+----+-------------+--------------------+-----------------+
|   N|       403807|              242889|60.14977452099642|
+----+-------------+--------------------+-----------------+

